In [1]:
import time

from rdflib.store import Store
from ordvev import OrdVev
start_time = time.time()
ordvev = OrdVev()
print ('done {0}s'.format(time.time() - start_time))

done 442.18004631996155s


In [2]:
import requests


from rdflib import Namespace
from rdflib import BNode
from rdflib.namespace import RDF, RDFS, OWL, DC, FOAF, XSD, SKOSD
from rdflib import plugin, Graph, Literal, URIRef

class CKAN:
    def __init__(self, url):
        self.url = url
        
    def get_datasets(self):
        req = requests.get('{0}/api/3/action/package_list'.format(self.url))
        json = req.json()
        return [requests.get('{0}/dataset/{1}.rdf'.format(self.url, d)) for d in json['result']]

graph = Graph()    
graph.parse('dcat-ap_1.1.rdf')
graph.parse('skos-odt.owl')
graph.parse('odt-v0.1.ttl', format='turtle')
graph.parse('autokey.ttl', format='turtle')
    
ckan = CKAN('http://78.91.98.234:5000')
ckan_datasets = ckan.get_datasets()
# Load datasets into the ontology
for dataset in ckan_datasets:
    graph.parse(data=dataset.text, format='xml')
graph.serialize(destination='output.ttl', format='turtle')

In [3]:
from rdflib import Namespace
from rdflib import BNode
from rdflib.namespace import RDF, RDFS, OWL, DC, FOAF, XSD, SKOS
from uuid import uuid4

ODT = Namespace('http://www.quaat.com/ontologies#')
DCAT = Namespace('http://www.w3.org/ns/dcat#')
DCT = Namespace('http://purl.org/dc/terms/')
ODTX = Namespace('http://www.quaat.com/ontology/ODTX#')
QEX = Namespace('http://www.quaat.com/extended_skos#')
WN20SCHEMA = Namespace('http://www.w3.org/2006/03/wn/wn20/schema/')
DN = Namespace('http://www.wordnet.dk/owl/instance/2009/03/instances/')
DN_SCHEMA = Namespace('http://www.wordnet.dk/owl/instance/2009/03/schema/')

# Helper method to create a link between a dataset and a concept with
# a given similarity-score
def add_similarity_link(graph, dataset, concept, score):
    uuid = uuid4().hex
    simlink = URIRef(QEX[uuid])
    graph.add((simlink, RDF.type, ODT.Similarity))
    graph.add((simlink, ODT.dataset, dataset))
    graph.add((simlink, ODT.concept, concept))
    graph.add((simlink, ODT.score, Literal(score, datatype=XSD.double)))
    return simlink

In [4]:
# Manual tagging (tag one or more concepts with the relevant dataset, then calculate the entire concept score vector)
add_similarity_link(graph, URIRef('http://78.91.98.234/dataset/0e3f86cf-2334-41f3-8762-935e5f83d638'), URIRef('http://www.quaat.com/ontologies#Road'), 1.0)
add_similarity_link(graph, URIRef('http://78.91.98.234/dataset/8c920c9b-77ad-4f64-8253-085078634e01'), URIRef('http://www.quaat.com/ontologies#TrafficInformation'), 1.0)
add_similarity_link(graph, URIRef('http://78.91.98.234/dataset/8c920c9b-77ad-4f64-8253-085078634e01'), URIRef('http://www.quaat.com/ontologies#WeatherCondition'), 1.0)
add_similarity_link(graph, URIRef('http://78.91.98.234/dataset/caefad21-41cc-41e6-9f59-e050486686ea'), URIRef('http://www.quaat.com/ontologies#Vehicle'), 1.0)
add_similarity_link(graph, URIRef('http://78.91.98.234/dataset/caefad21-41cc-41e6-9f59-e050486686ea'), URIRef('http://www.quaat.com/ontologies#Region'), 1.0)
add_similarity_link(graph, URIRef('http://78.91.98.234/dataset/0e3f86cf-2334-41f3-8762-935e5f83d638'), URIRef('http://www.quaat.com/ontologies#Region'), 1.0)
add_similarity_link(graph, URIRef('http://78.91.98.234/dataset/0e3f86cf-2334-41f3-8762-935e5f83d638'), URIRef('http://www.quaat.com/ontologies#BusTerminal'), 1.0)
add_similarity_link(graph, URIRef('http://78.91.98.234/dataset/8c920c9b-77ad-4f64-8253-085078634e01'), URIRef('http://www.quaat.com/ontologies#TransportNetworkCondition'), 1.0)


rdflib.term.URIRef('http://www.quaat.com/extended_skos#d445b020c5884aa1a99208b78ce80abd')

In [5]:
# Automatic tagging (extract keywords from the datasets and calculate the entire concept score vector)

In [6]:
# Display concept-vectors
concepts = [c for c in graph.subjects(RDF.type, SKOS.Concept)]
datasets = [d for d in graph.subjects(RDF.type, DCAT.Dataset)]
similarities = [s for s in graph.subjects(RDF.type, ODT.Similarity)] 
dataset_concept_vectors = {}

for dataset in datasets:
    concept_vector = dict(zip(concepts, [0.0]*len(concepts)))
    dataset_concept_vectors.update({dataset: concept_vector})

for s in similarities:
    dataset = next(graph.objects(s, ODT.dataset))    
    concept = next(graph.objects(s, ODT.concept))
    score = float(str(next(graph.objects(s, ODT.score))))
    dataset_concept_vectors[dataset][concept] = max(dataset_concept_vectors[dataset][concept], score)    

print (len(dataset_concept_vectors))

# Use the concept-concept-similarity (CCS) table to compute the adjusted concept vectors

220


In [7]:
print (dataset_concept_vectors[URIRef('http://78.91.98.234/dataset/0e3f86cf-2334-41f3-8762-935e5f83d638')])

# Adjust the ontology based on the semantic similarity of the concepts. Keep the highest scores
from odt.navigate import GraphNavigate

def semantic_adjustment(graph):
    nav = GraphNavigate(graph)
    

{rdflib.term.URIRef('http://www.quaat.com/ontologies#Tunnel'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Junction'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Emission'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Facility'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Roadway'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Building'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Collection'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#TrafficQueue'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Highway'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Zone'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Speedway'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Vehicle'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#Organization'): 0.0, rdflib.term.URIRef('http://www.quaat.com/ontologies#WeatherForecast'): 0.0, rdflib.term.URIRef('http:

In [8]:
from odt.skosnavigate import SKOSNavigate

print ('sum_wup ', ordvev.sim_wup('katt', 'marsvin'))
nav = SKOSNavigate(graph)
c1 = URIRef('http://www.quaat.com/ontologies#Road')
c2 = URIRef('http://www.quaat.com/ontologies#TrafficInformation')
print (nav.sim_wup(c1, c2))
print (nav.sim_wup(c1, c1))

sum_wup  0.7777777777777778
0.2
1.0


In [9]:
import pandas as pd
ccs_matrix = []
for c1 in concepts:
    ccs_inner = []
    for c2 in concepts:
        score = nav.sim_wup(c1, c2)
        ccs_inner.append(score)
    ccs_matrix.append(ccs_inner)

ccs = pd.DataFrame(columns=concepts, index=concepts, data=ccs_matrix)
ccs

,http://www.quaat.com/ontologies#Tunnel,http://www.quaat.com/ontologies#Junction,http://www.quaat.com/ontologies#Emission,http://www.quaat.com/ontologies#Facility,http://www.quaat.com/ontologies#Roadway,http://www.quaat.com/ontologies#Building,http://www.quaat.com/ontologies#Collection,http://www.quaat.com/ontologies#TrafficQueue,http://www.quaat.com/ontologies#Highway,http://www.quaat.com/ontologies#Zone,...,http://www.quaat.com/ontologies#Airport,http://www.quaat.com/ontologies#Toolbooth,http://www.quaat.com/ontologies#Booth,http://www.quaat.com/ontologies#Action,http://www.quaat.com/ontologies#Artifact,http://www.quaat.com/ontologies#Communication,http://www.quaat.com/ontologies#BusTerminal,http://www.quaat.com/ontologies#Intersection,http://www.quaat.com/ontologies#Entity,http://www.quaat.com/ontologies#Terminal
http://www.quaat.com/ontologies#Tunnel,1.000000,0.400000,0.222222,0.666667,0.727273,0.600000,0.222222,0.200000,0.727273,0.400000,...,0.600000,0.545455,0.600000,0.222222,0.750000,0.250000,0.545455,0.363636,0.333333,0.600000
http://www.quaat.com/ontologies#Junction,0.400000,1.000000,0.222222,0.444444,0.363636,0.400000,0.222222,0.200000,0.363636,0.800000,...,0.400000,0.363636,0.400000,0.222222,0.500000,0.250000,0.363636,0.909091,0.333333,0.400000
http://www.quaat.com/ontologies#Emission,0.222222,0.222222,1.000000,0.250000,0.200000,0.222222,0.250000,0.222222,0.200000,0.222222,...,0.222222,0.200000,0.222222,0.500000,0.285714,0.285714,0.200000,0.200000,0.400000,0.222222
http://www.quaat.com/ontologies#Facility,0.666667,0.444444,0.250000,1.000000,0.600000,0.666667,0.250000,0.222222,0.600000,0.444444,...,0.888889,0.600000,0.666667,0.250000,0.857143,0.285714,0.800000,0.400000,0.400000,0.888889
http://www.quaat.com/ontologies#Roadway,0.727273,0.363636,0.200000,0.600000,1.000000,0.545455,0.200000,0.181818,0.833333,0.363636,...,0.545455,0.500000,0.545455,0.200000,0.666667,0.222222,0.500000,0.333333,0.285714,0.545455
http://www.quaat.com/ontologies#Building,0.600000,0.400000,0.222222,0.666667,0.545455,1.000000,0.222222,0.200000,0.545455,0.400000,...,0.600000,0.727273,0.800000,0.222222,0.750000,0.250000,0.545455,0.363636,0.333333,0.600000
http://www.quaat.com/ontologies#Collection,0.222222,0.222222,0.250000,0.250000,0.200000,0.222222,1.000000,0.666667,0.200000,0.222222,...,0.222222,0.200000,0.222222,0.250000,0.285714,0.571429,0.200000,0.200000,0.400000,0.222222
http://www.quaat.com/ontologies#TrafficQueue,0.200000,0.200000,0.222222,0.222222,0.181818,0.200000,0.666667,1.000000,0.181818,0.200000,...,0.200000,0.181818,0.200000,0.222222,0.250000,0.500000,0.181818,0.181818,0.333333,0.200000
http://www.quaat.com/ontologies#Highway,0.727273,0.363636,0.200000,0.600000,0.833333,0.545455,0.200000,0.181818,1.000000,0.363636,...,0.545455,0.500000,0.545455,0.200000,0.666667,0.222222,0.500000,0.333333,0.285714,0.545455
http://www.quaat.com/ontologies#Zone,0.400000,0.800000,0.222222,0.444444,0.363636,0.400000,0.222222,0.200000,0.363636,1.000000,...,0.400000,0.363636,0.400000,0.222222,0.500000,0.250000,0.363636,0.727273,0.333333,0.400000


In [10]:
import numpy as np

ds = []
f = dict(zip(concepts, [0.0]*len(list(concepts))))
ds.append(f)
cds = pd.DataFrame(columns=concepts, index=datasets, data=ds)

for s in similarities:
    dataset = next(graph.objects(s, ODT.dataset), None)
    concept = next(graph.objects(s, ODT.concept), None)
    score = float(str(next(graph.objects(s, ODT.score), '0.0')))
    
    for c in concepts:
        weighted = score * ccs[concept][c]        
        cds[c][dataset] = max(cds[c][dataset], weighted)

#dataset = URIRef('http://78.91.98.234/dataset/0e3f86cf-2334-41f3-8762-935e5f83d638')
#cds.loc[dataset] > 0.5
cds



,http://www.quaat.com/ontologies#Tunnel,http://www.quaat.com/ontologies#Junction,http://www.quaat.com/ontologies#Emission,http://www.quaat.com/ontologies#Facility,http://www.quaat.com/ontologies#Roadway,http://www.quaat.com/ontologies#Building,http://www.quaat.com/ontologies#Collection,http://www.quaat.com/ontologies#TrafficQueue,http://www.quaat.com/ontologies#Highway,http://www.quaat.com/ontologies#Zone,...,http://www.quaat.com/ontologies#Airport,http://www.quaat.com/ontologies#Toolbooth,http://www.quaat.com/ontologies#Booth,http://www.quaat.com/ontologies#Action,http://www.quaat.com/ontologies#Artifact,http://www.quaat.com/ontologies#Communication,http://www.quaat.com/ontologies#BusTerminal,http://www.quaat.com/ontologies#Intersection,http://www.quaat.com/ontologies#Entity,http://www.quaat.com/ontologies#Terminal
http://78.91.98.234/dataset/6370c400-1fd2-45b0-a018-1362b7303088,0.545455,0.681818,0.375000,0.750000,0.625000,0.666667,0.750000,0.500000,0.625000,0.545455,...,0.666667,0.600000,0.666667,0.562500,0.642857,0.666667,0.600000,0.625000,0.380952,0.666667
http://78.91.98.234/dataset/aad23951-b6eb-489e-8bfe-59949eed916a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
http://78.91.98.234/dataset/78a0cc73-b4de-4c87-84b0-c5bb7ba079c2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
http://78.91.98.234/dataset/92a9b284-6073-47e9-8eee-5f2195d0879a,0.606061,0.562500,0.342857,0.500000,0.694444,0.533333,0.600000,0.400000,0.694444,0.562500,...,0.454545,0.606061,0.666667,0.500000,0.555556,0.545454,0.416667,0.500000,0.380952,0.454545
http://78.91.98.234/dataset/0e3f86cf-2334-41f3-8762-935e5f83d638,0.800000,0.666667,0.250000,0.800000,0.909091,0.600000,0.250000,0.222222,0.909091,0.666667,...,0.727273,0.545455,0.600000,0.250000,0.750000,0.285714,1.000000,0.600000,0.400000,0.909091
http://78.91.98.234/dataset/8c01967c-4cdb-40ef-8e97-1fdd0430aaa7,0.528926,0.661157,0.444444,0.545454,0.606061,0.484848,0.666667,0.500000,0.606061,0.528926,...,0.484848,0.436364,0.484848,0.666667,0.527473,0.800000,0.436364,0.606061,0.444444,0.484848
http://78.91.98.234/dataset/52d0d63e-d26c-43b9-91e2-13b3b2a96c89,0.545455,0.681818,0.375000,0.500000,0.625000,0.444444,0.750000,0.500000,0.625000,0.545455,...,0.444444,0.400000,0.444444,0.562500,0.500000,0.666667,0.400000,0.625000,0.380952,0.444444
http://78.91.98.234/dataset/8113c645-840b-4cb3-adec-ffdcf7d1c07b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
http://78.91.98.234/dataset/ab728316-fc8f-47fb-a111-3d3851d9a5cb,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
http://78.91.98.234/dataset/e753aea4-7857-46c7-a385-ff3cfb7da823,0.480000,0.326531,0.190476,0.571429,0.436364,0.800000,0.666667,0.666667,0.436364,0.326531,...,0.507937,0.581818,0.640000,0.190476,0.600000,0.444444,0.533333,0.290909,0.380952,0.533333


In [15]:
cds.to_json(path_or_buf='nb_dcs_output.csv')
ccs.to_json(path_or_buf='nb_ccs_output.csv')

In [12]:
graph.serialize(destination='autotag.ttl', format='turtle')

In [13]:
import pandas as pd
distros = graph.subjects(RDF.type, DCAT.Distribution)

#for dist in distros:    
#    file_format = next(graph.objects(dist, DCT['format']), None)
#    print (dist, file_format)
    


# setup correct similarities for the NetEX format
def classify_netex(graph, dataset):        
    add_similarity_link(graph, dataset, URIRef('http://www.quaat.com/ontologies#TrafficInformation'), 1.0)
    add_similarity_link(graph, dataset, URIRef('http://www.quaat.com/ontologies#BusTerminal'), 1.0)


def classify_csv(graph, dataset, url):
    try:
        c = pd.read_csv(url, nrows=2) # Just read the first 2 rows
       # print (list(c.columns.values))
    except:
        pass
    
# Find distributions
files = set([])
for dataset in datasets:
    distributions = graph.objects(dataset, DCAT.distribution)
    for distribution in distributions:    
        file_format = next(graph.objects(distribution, DCT['format']), None)
        accessUrl = next(graph.objects(distribution, DCAT['accessURL']), None)
        if file_format and accessUrl:
            
            if str(file_format).lower() == "csv":                
                pass
                #classify_csv(graph, dataset, str(accessUrl))
            
            elif str(file_format).lower() == "netex":
                classify_netex(graph, dataset)
                
